In [6]:
from jax import random
import parabellum as pb

In [7]:
place = "Indre By, Copenhagen, Denmark"
terrain = pb.terrain_fn(place, 1000)
scenario = pb.make_scenario(place, terrain, 10, 10)
env = pb.Environment(scenario)

In [8]:
rng, key = random.split(random.PRNGKey(seed := 0))
obs, state = env.reset(key)
state_seq = []

In [9]:
for i in range(100):
    rng, act_rng, step_key = random.split(rng, 3)
    act_key = random.split(act_rng, len(env.agents))
    actions = {
        a: 0 * env.action_space(a).sample(k) for a, k in zip(env.agents, act_key)
    }
    state_seq.append((step_key, state, actions))
    obs, state, reward, done, info = env.step(step_key, state, actions)

In [10]:
vis = pb.Visualizer(env, state_seq)
vis.animate()

800it [00:13, 60.61it/s]                        


Moviepy - Building video output/parabellum.mp4.
Moviepy - Writing video output/parabellum.mp4



Moviepy - Done !
Moviepy - video ready output/parabellum.mp4
